In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define the pretrained model (choose from 'alexnet', 'vgg16', 'resnet18', 'resnet50', 'resnet101', etc.)
pretrained_model_name = 'resnet18'

# Load pretrained model
pretrained_model = getattr(models, pretrained_model_name)(pretrained=True)

# Modify the model for SVHN classification
num_classes = 10  # SVHN has 10 classes (digits 0-9)
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, num_classes)

# Load SVHN dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match pretrained model input size
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])

svhn_train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
svhn_test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(svhn_train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(svhn_test_dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model.to(device)

for epoch in range(num_epochs):
    pretrained_model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = pretrained_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(svhn_train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Evaluate the model
pretrained_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = pretrained_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")


/home/avi06/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/avi06/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/avi06/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
52.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Curr

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Epoch [1/10], Loss: 0.3407
Epoch [2/10], Loss: 0.2162
Epoch [3/10], Loss: 0.1749
Epoch [4/10], Loss: 0.1471
Epoch [5/10], Loss: 0.1164
Epoch [6/10], Loss: 0.0912
Epoch [7/10], Loss: 0.0675
